In [14]:
import cv2

# Load the Haar cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # or try 1, 2, etc.

if not cap.isOpened():
    print("❌ Cannot open camera.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret or frame is None:
        print("❌ Failed to grab frame.")
        break

    cv2.imshow("Live Camera", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC to exit
        break

cap.release()
cv2.destroyAllWindows()




while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect face(s)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        # Draw face box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Focus on upper half of face (eye region)
        roi_gray = gray[y:y + h//2, x:x + w]
        _, thresh = cv2.threshold(roi_gray, 50, 255, cv2.THRESH_BINARY_INV)

        contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if 50 < area < 200:
                x2, y2, w2, h2 = cv2.boundingRect(cnt)
                cv2.rectangle(frame, (x + x2, y + y2), (x + x2 + w2, y + y2 + h2), (255, 0, 0), 1)

    cv2.imshow('Face + Eye Region Contours', frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC to quit
        break

cap.release()
cv2.destroyAllWindows()


❌ Failed to grab frame.
Failed to grab frame.


In [15]:
import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]

        # Convert to HSV for skin detection
        hsv = cv2.cvtColor(face_roi, cv2.COLOR_BGR2HSV)

        # Skin color range in HSV (tweak as needed for your lighting)
        lower_skin = np.array([0, 30, 60], dtype=np.uint8)
        upper_skin = np.array([20, 150, 255], dtype=np.uint8)
        skin_mask = cv2.inRange(hsv, lower_skin, upper_skin)

        # Optional: smooth and fill holes
        skin_mask = cv2.erode(skin_mask, None, iterations=2)
        skin_mask = cv2.dilate(skin_mask, None, iterations=2)
        skin_mask = cv2.GaussianBlur(skin_mask, (3, 3), 0)

        # Find largest contour (likely the face region)
        contours, _ = cv2.findContours(skin_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            largest = max(contours, key=cv2.contourArea)
            cv2.drawContours(face_roi, [largest], -1, (0, 255, 0), 2)

    cv2.imshow("Face Shape Contour", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [7]:
import cv2
import dlib
import numpy as np

# Load detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("C:/Users/ryker/CNN-Notebook/shape_predictor_68_face_landmarks.dat")
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)

        points = []
        for n in range(0, 68):  # All landmark points
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            points.append((x, y))
            cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)

        # Optional: Draw a convex hull around outer face contour
        hull_points = np.array(points[0:17])  # Jawline only
        cv2.polylines(frame, [hull_points], isClosed=False, color=(255, 0, 0), thickness=2)

    cv2.imshow("Landmark Face Shape", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


RuntimeError: Unsupported image type, must be 8bit gray or RGB image.

In [30]:
import cv2
import dlib
import numpy as np

# Load models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("C:/Users/ryker/CNN-Notebook/shape_predictor_68_face_landmarks.dat")
connections = [
    # Jawline
    *[(i, i+1) for i in range(0, 16)],
    # Left eyebrow
    *[(i, i+1) for i in range(17, 21)],
    # Right eyebrow
    *[(i, i+1) for i in range(22, 26)],
    # Nose bridge
    *[(i, i+1) for i in range(27, 30)],
    # Nose bottom
    *[(i, i+1) for i in range(30, 35)],
    (35, 30),
    # Left eye
    *[(36,37), (37,38), (38,39), (39,36)],
    *[(36,41), (41,40), (40,39)],
    # Right eye
    *[(42,43), (43,44), (44,45), (45,42)],
    *[(42,47), (47,46), (46,45)],
    # Outer lips
    *[(i, i+1) for i in range(48, 59)],
    (59, 48),
    # Inner lips
    *[(i, i+1) for i in range(60, 67)],
    (67, 60)
]

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
if not cap.isOpened():
    print("cant open camera.")
    exit()
while True:
    ret, frame = cap.read()
    if not ret or frame is None:
        print("Failed to fetch the frame")
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    print("gray dtype:", gray.dtype, "shape:", gray.shape)  # Debug print

    if gray.dtype != np.uint8:
        gray = gray.astype(np.uint8)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)
        points = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(68)]

        for x, y in points:
            cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)

        for i, j in connections:
            pt1 = points[i]
            pt2 = points[j]
            cv2.line(frame, pt1, pt2, (255, 255, 0), 1)
            #cv2.putText(frame, f"{i}", (pt1[0], pt1[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)


    cv2.imshow("Geometric Face Map", frame)

    if cv2.waitKey(1) & 0xFF == 27: # (this is the escape key)
        break

cap.release()
cv2.destroyAllWindows()


gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (480, 640)
gray dtype: uint8 shape: (48

In [33]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
if not cap.isOpened():
    print("cant open camera.")
    exit()
while True:
    ret, frame = cap.read()
    if not ret or frame is None:
        print("Failed to fetch the frame")
        break
    green = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    if green.dtype != np.uint8:
        green = green.astype(np.uint8)
    tennis_ball = cv2.inRange(green, np.array([25, 75, 85]), np.array([50, 220, 255]))
    tennis_ball = cv2.GaussianBlur(tennis_ball, (5, 5), 0)
    tennis_ball = cv2.morphologyEx(tennis_ball, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))
    contours, _ = cv2.findContours(tennis_ball, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if 1000 < area < 10000:
            (x,y),radius = cv2.minEnclosingCircle(cnt)
            if radius < 10:
                continue
            center = (int(x),int(y))
            cv2.circle(frame, center, 2, (0, 0, 255), 3)
            x, y = int(x), int(y)
            radius = int(radius)
            cv2.circle(frame, center, radius, (0, 255, 0), 2)
            cv2.putText(frame, "Tennis Ball", (int(x-radius), int(y-radius)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.imshow("Tennis Ball Detection", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to exit
        break
cap.release()
cv2.destroyAllWindows() 



In [28]:
import cv2
import dlib

cap = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()

while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    print("gray dtype:", gray.dtype, "shape:", gray.shape)
    faces = detector(gray)
    for face in faces:
        cv2.rectangle(frame, (face.left(), face.top()), (face.right(), face.bottom()), (0,255,0), 2)
    cv2.imshow("dlib test", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()

gray dtype: uint8 shape: (480, 640)


RuntimeError: Unsupported image type, must be 8bit gray or RGB image.

In [9]:
import cv2
import numpy as np

# Load OpenCV's face cascade (comes with OpenCV installation)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# If you have the dlib landmark file, we can try to use it once dlib is fixed
# For now, we'll just do face detection
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
if not cap.isOpened():
    print("Can't open camera.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret or frame is None:
        print("Failed to fetch the frame")
        break
    
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    
    # Draw rectangles around faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Draw a simple face outline (not as detailed as dlib landmarks)
        center = (x + w//2, y + h//2)
        cv2.circle(frame, center, 2, (0, 255, 0), -1)
        
        # Draw some basic points
        # Eyes (approximate positions)
        left_eye = (x + w//4, y + h//3)
        right_eye = (x + 3*w//4, y + h//3)
        cv2.circle(frame, left_eye, 3, (0, 255, 0), -1)
        cv2.circle(frame, right_eye, 3, (0, 255, 0), -1)
        
        # Nose
        nose = (x + w//2, y + h//2)
        cv2.circle(frame, nose, 2, (0, 255, 0), -1)
        
        # Mouth
        mouth = (x + w//2, y + 2*h//3)
        cv2.circle(frame, mouth, 3, (0, 255, 0), -1)

    cv2.imshow("Face Detection (OpenCV)", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC key
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [4]:
import cv2
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Cannot open camera.")
else:
    ret, frame = cap.read()
    print("ret:", ret, "frame type:", type(frame))
    cap.release()

ret: True frame type: <class 'numpy.ndarray'>


In [5]:
import cv2
import dlib
import numpy as np


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("C:/Users/ryker/CNN-Notebook/shape_predictor_68_face_landmarks.dat")

# make face hav etriangles
def get_delaunay_triangles(rect, points):
    subdiv = cv2.Subdiv2D(rect)
    for p in points:
        subdiv.insert(p)   
    triangle_list = subdiv.getTriangleList()
    triangles = []

    for t in triangle_list:
        pts = [(int(t[0]), int(t[1])), (int(t[2]), int(t[3])), (int(t[4]), int(t[5]))]
        idx = []
        for pt in pts:
            for i, p in enumerate(points):
                if abs(pt[0] - p[0]) < 2 and abs(pt[1] - p[1]) < 2:
                    idx.append(i)
        if len(idx) == 3:
            triangles.append(tuple(idx))
    return triangles

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)
        points = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(68)]

        rect = (0, 0, width, height)
        triangles = get_delaunay_triangles(rect, points)
        for tri in triangles:
            pt1 = points[tri[0]]
            pt2 = points[tri[1]]
            pt3 = points[tri[2]]
            cv2.line(frame, pt1, pt2, (0, 255, 255), 1)
            cv2.line(frame, pt2, pt3, (0, 255, 255), 1)
            cv2.line(frame, pt3, pt1, (0, 255, 255), 1)
        #draw the points
        for x, y in points:
            cv2.circle(frame, (x, y), 1, (0, 100, 255), -1)

    cv2.imshow("Face Mesh Grid (Delaunay)", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


RuntimeError: Unsupported image type, must be 8bit gray or RGB image.

In [10]:
import cv2
import mediapipe as mp

# Initialize MediaPipe
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip for natural selfie view
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Draw landmarks and connections
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2),
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=3)
            )

            # Optional: Print fingertip points
            fingertips = [4, 8, 12, 16, 20]  # Thumb, Index, Middle, Ring, Pinky tips
            h, w, _ = frame.shape
            for i in fingertips:
                x = int(hand_landmarks.landmark[i].x * w)
                y = int(hand_landmarks.landmark[i].y * h)
                cv2.circle(frame, (x, y), 6, (0, 0, 255), -1)
                cv2.putText(frame, f'{i}', (x+5, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)

    cv2.imshow("Hand Tracker", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [5]:
import cv2
import mediapipe as mp

# MediaPipe modules
mp_face = mp.solutions.face_mesh
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Setup
face_mesh = mp_face.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.7)
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7)

# Drawing specs
face_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1, color=(255, 0, 0))
face_conn_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1, color=(0, 255, 0))

hand_spec = mp_drawing.DrawingSpec(thickness=2, circle_radius=3, color=(0, 255, 255))
hand_conn_spec = mp_drawing.DrawingSpec(thickness=2, circle_radius=2, color=(255, 0, 255))

# Camera
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    face_results = face_mesh.process(rgb)
    hand_results = hands.process(rgb)

    # Draw face mesh
    if face_results.multi_face_landmarks:
        for face_landmarks in face_results.multi_face_landmarks:
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=face_landmarks,
                connections=mp_face.FACEMESH_CONTOURS,  # try FACEMESH_TESSELATION for full grid
                landmark_drawing_spec=face_spec,
                connection_drawing_spec=face_conn_spec
            )

    # Draw hand keypoints
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                hand_spec,
                hand_conn_spec
            )

    cv2.imshow("Face and Hand Tracker", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


SyntaxError: invalid syntax (4027268105.py, line 1)